In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression


!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 데이터 로드
df = pd.read_csv('시군구별전력사용량_202408.csv', encoding='cp949')  # encoding='cp949' 추가

# 필요한 열 선택 및 조건 필터링
df = df[['시도', '시군구', '계약종별', '1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월']]
df = df[df['계약종별'] == '합 계']

# 결과 저장할 리스트
results = []

for index, row in df.iterrows():
    # 1월부터 7월까지 데이터를 사용하여 8월 예측
    X = [[1], [2], [3], [4],[5],[6],[7]]  # 월
    y = [row['1월'], row['2월'], row['3월'], row['4월'], row['5월'], row['6월'], row['7월']]
    model = LinearRegression()
    model.fit(X, y)

    # 8월 예측
    predicted_may = model.predict([[8]])[0]

    # 실제 8월 값
    actual_may = row['8월']

    # 결과 저장
    results.append({
        '시도': row['시도'],
        '시군구': row['시군구'],
        '예측값': np.floor(predicted_may),
        '실제값': actual_may,
        '오차': np.floor(abs(predicted_may - actual_may))
    })

# 결과를 DataFrame으로 변환
result_df = pd.DataFrame(results)

# 결과 출력 또는 저장
result_df
# result_df.to_csv('prediction_results.csv', index=False)


import pandas as pd
import matplotlib.pyplot as plt

# 한글 폰트 설정
plt.rcParams['font.family'] = 'NanumGothic'
# plt.rcParams['font.family'] = 'AppleGothic'  # MacOS
# plt.rcParams['font.family'] = 'NanumGothic'  # Ubuntu

# Calculate the absolute error and mean squared error for each province
result_df['절대오차'] = abs(result_df['예측값'] - result_df['실제값'])
result_df['제곱오차'] = (result_df['예측값'] - result_df['실제값']) ** 2

# MSE 계산
mse_by_province = result_df.groupby('시도')['제곱오차'].mean()
mae_by_province = result_df.groupby('시도')['절대오차'].mean()

# 그래프 시각화
plt.figure(figsize=(12, 6))

# MSE 그래프
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
mse_by_province.plot(kind='bar', color='orange', alpha=0.7)
plt.xlabel('시도')
plt.ylabel('평균 제곱 오차 MSE')
plt.title('시도별 평균 제곱 오차 MSE')
plt.xticks(rotation=45, ha='right')

# MAE 그래프
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
mae_by_province.plot(kind='bar', color='skyblue', alpha=0.7)
plt.xlabel('시도')
plt.ylabel('평균 절대 오차 MAE')
plt.title('시도별 평균 절대 오차 MAE')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()  # Adjust layout to prevent labels from overlapping
plt.show()


In [ ]:
# 작년의 사용량을 바탕으로 올해를 예측해봄
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# 데이터 로드
df_after = pd.read_csv('시군구별전력사용량2023.csv', encoding='cp949')  # 작년 데이터
df = pd.read_csv('시군구별전력사용량_202408.csv', encoding='cp949')  # 현재 데이터

# 필요한 열 선택 및 조건 필터링
df = df[['시도', '시군구', '계약종별', '1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월']]
df_after = df_after[['시도', '시군구', '계약종별', '1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월']]
df = df[df['계약종별'] == '합 계']
df_after = df_after[df_after['계약종별'] == '합 계']

# 결과 저장할 리스트
results = []

# 시도와 시군구를 기준으로 데이터 병합
merged_df = pd.merge(df, df_after, on=['시도', '시군구'], suffixes=('_current', '_last'))

for index, row in merged_df.iterrows():
    # 현재 연도와 작년 데이터를 결합하여 예측
    X = [[1], [2], [3], [4], [5], [6], [7]]  # 월을 나타내는 변수
    y = [
        row['1월_current'], row['2월_current'], row['3월_current'],
        row['4월_current'], row['5월_current'], row['6월_current'], row['7월_current']
    ]
    y_last = [
        row['1월_last'], row['2월_last'], row['3월_last'],
        row['4월_last'], row['5월_last'], row['6월_last'], row['7월_last']
    ]

    # 현재 데이터와 작년 데이터 평균
    y_avg = [(a + b) / 2 for a, b in zip(y, y_last)]

    # 회귀 모델 학습
    model = LinearRegression()
    model.fit(X, y_avg)

    # 8월 예측
    predicted_aug = model.predict([[8]])[0]

    # 실제 8월 값 및 작년 8월 값
    actual_aug = row['8월_current']
    last_year_aug = row['8월_last']

    # 결과 저장
    results.append({
        '시도': row['시도'],
        '시군구': row['시군구'],
        '예측값': np.floor(predicted_aug),
        '실제값': actual_aug,
        '작년실제값': last_year_aug,
        '오차': np.floor(abs(predicted_aug - actual_aug))
    })

# 결과를 DataFrame으로 변환
result_df = pd.DataFrame(results)

# 결과 출력 또는 저장
result_df
# result_df.to_csv('prediction_results.csv', index=False)
